Run a simple test using the PseudoCrabMinimal benchmark. Note that the sum of the
parameters needs to equal 1.0.

In [15]:
%load_ext autoreload
%autoreload 2
from matsci_opt_benchmarks.crabnet_hyperparameter.core import PseudoCrabMinimal

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
pc = PseudoCrabMinimal(dummy=True)

In [17]:
results = pc.evaluate({"x1": 0.5, "x2": 0.2, "x3": 0.3}, dummy=True)

1.0
user parameters are : {'pos_scaler': 0.5, 'weight_decay': 0.2, 'train_frac': 0.307}
{'N': 3, 'alpha': 0.5, 'd_model': 512, 'dim_feedforward': 2048, 'dropout': 0.1, 'emb_scaler': 1.0, 'eps': 1e-06, 'epochs_step': 10, 'fudge': 0.02, 'heads': 4, 'k': 6, 'lr': 0.001, 'pe_resolution': 5000, 'ple_resolution': 5000, 'pos_scaler': 0.5, 'weight_decay': 0.2, 'batch_size': 32, 'out_hidden4': 128, 'betas1': 0.9, 'betas2': 0.999, 'train_frac': 0.307, 'bias': False, 'criterion': 'RobustL1', 'elem_prop': 'mat2vec'}


In [18]:
results

{'mae': 0.4756186025567695}

Next, we'll use [Ax](https://ax.dev) to minimize the MAE. We'll reparameterize the
following constraint:
$x_1 + x_2 + x_3 = 1.0$

into:

$x_1 + x_2 \le 1.0$ and $x_3 = x_1 + x_2 - 1.0$

and remove $x_3$ from the search space by calculating it from $x_1$ and $x_2$ in the
`evaluate` function.

In [19]:
from ax.service.ax_client import AxClient

dummy = True

def evaluate(parameters):
    parameters["x3"] = 1 - (parameters["x1"] + parameters["x2"])
    parameters["x3"] = max(0.0, parameters["x3"])
    return pc.evaluate(parameters, dummy=dummy)

ax_client = AxClient()
ax_client.create_experiment(
    name="pseudo_crab",
    parameters=[
        {
            "name": "x1",
            "type": "range",
            "bounds": [0.0, 1.0],
            "value_type": "float",
        },
        {
            "name": "x2",
            "type": "range",
            "bounds": [0.0, 1.0],
        },
    ],
    objective_name="mae",
    parameter_constraints=["x1 + x2  <= 1.0"],
    minimize=True,
)

for _ in range(15):
    parameters, trial_index = ax_client.get_next_trial()
    ax_client.complete_trial(
        trial_index=trial_index,
        raw_data=evaluate(parameters),
    )

best_parameters, metrics = ax_client.get_best_parameters()

[INFO 03-21 22:56:59] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 03-21 22:56:59] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x2. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-21 22:56:59] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='x1', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='x2', parameter_type=FLOAT, range=[0.0, 1.0])], parameter_constraints=[ParameterConstraint(1.0*x1 + 1.0*x2 <= 1.0)]).
[INFO 03-21 22:56:59] ax.modelbridge.dispatch_utils: Using Bayesian optimization since there are more ordered parameters than there are categories for the unordered categorical parameters.
[INFO 03-21 22:56:59] ax.modelbridge

1.0
user parameters are : {'pos_scaler': 0.11113402, 'weight_decay': 0.7879766, 'train_frac': 0.10988047}
{'N': 3, 'alpha': 0.5, 'd_model': 512, 'dim_feedforward': 2048, 'dropout': 0.1, 'emb_scaler': 1.0, 'eps': 1e-06, 'epochs_step': 10, 'fudge': 0.02, 'heads': 4, 'k': 6, 'lr': 0.001, 'pe_resolution': 5000, 'ple_resolution': 5000, 'pos_scaler': 0.11113402, 'weight_decay': 0.7879766, 'batch_size': 32, 'out_hidden4': 128, 'betas1': 0.9, 'betas2': 0.999, 'train_frac': 0.10988047, 'bias': False, 'criterion': 'RobustL1', 'elem_prop': 'mat2vec'}


[INFO 03-21 22:57:00] ax.service.ax_client: Completed trial 0 with data: {'mae': (0.737133, None)}.
[INFO 03-21 22:57:00] ax.service.ax_client: Generated new trial 1 with parameters {'x1': 0.347143, 'x2': 0.558201}.


1.0
user parameters are : {'pos_scaler': 0.34714317, 'weight_decay': 0.55820143, 'train_frac': 0.103708796}
{'N': 3, 'alpha': 0.5, 'd_model': 512, 'dim_feedforward': 2048, 'dropout': 0.1, 'emb_scaler': 1.0, 'eps': 1e-06, 'epochs_step': 10, 'fudge': 0.02, 'heads': 4, 'k': 6, 'lr': 0.001, 'pe_resolution': 5000, 'ple_resolution': 5000, 'pos_scaler': 0.34714317, 'weight_decay': 0.55820143, 'batch_size': 32, 'out_hidden4': 128, 'betas1': 0.9, 'betas2': 0.999, 'train_frac': 0.103708796, 'bias': False, 'criterion': 'RobustL1', 'elem_prop': 'mat2vec'}


[INFO 03-21 22:57:00] ax.service.ax_client: Completed trial 1 with data: {'mae': (0.77037, None)}.
[INFO 03-21 22:57:00] ax.service.ax_client: Generated new trial 2 with parameters {'x1': 0.150765, 'x2': 0.360584}.


...

[INFO 03-21 22:58:34] ax.service.ax_client: Generated new trial 13 with parameters {'x1': 0.074174, 'x2': 0.0}.
[INFO 03-21 22:58:34] ax.service.ax_client: Completed trial 13 with data: {'mae': (0.381291, None)}.


1.0
user parameters are : {'pos_scaler': 0.07417395, 'weight_decay': 0.0, 'train_frac': 0.9265678}
{'N': 3, 'alpha': 0.5, 'd_model': 512, 'dim_feedforward': 2048, 'dropout': 0.1, 'emb_scaler': 1.0, 'eps': 1e-06, 'epochs_step': 10, 'fudge': 0.02, 'heads': 4, 'k': 6, 'lr': 0.001, 'pe_resolution': 5000, 'ple_resolution': 5000, 'pos_scaler': 0.07417395, 'weight_decay': 0.0, 'batch_size': 32, 'out_hidden4': 128, 'betas1': 0.9, 'betas2': 0.999, 'train_frac': 0.9265678, 'bias': False, 'criterion': 'RobustL1', 'elem_prop': 'mat2vec'}


[INFO 03-21 22:58:43] ax.service.ax_client: Generated new trial 14 with parameters {'x1': 0.0, 'x2': 0.0}.


1.0
user parameters are : {'pos_scaler': 0.0, 'weight_decay': 0.0, 'train_frac': 1.0}
{'N': 3, 'alpha': 0.5, 'd_model': 512, 'dim_feedforward': 2048, 'dropout': 0.1, 'emb_scaler': 1.0, 'eps': 1e-06, 'epochs_step': 10, 'fudge': 0.02, 'heads': 4, 'k': 6, 'lr': 0.001, 'pe_resolution': 5000, 'ple_resolution': 5000, 'pos_scaler': 0.0, 'weight_decay': 0.0, 'batch_size': 32, 'out_hidden4': 128, 'betas1': 0.9, 'betas2': 0.999, 'train_frac': 1.0, 'bias': False, 'criterion': 'RobustL1', 'elem_prop': 'mat2vec'}


[INFO 03-21 22:58:44] ax.service.ax_client: Completed trial 14 with data: {'mae': (0.381499, None)}.
[WARNING 03-21 22:58:45] ax.modelbridge.cross_validation: Metric mae was unable to be reliably fit.
[WARNING 03-21 22:58:45] ax.service.utils.best_point: Model fit is poor; falling back on raw data for best point.
[WARNING 03-21 22:58:45] ax.service.utils.best_point: Model fit is poor and data on objective metric mae is noisy; interpret best points results carefully.


In [ ]:
best_parameters, metrics

({'x1': 0.1748458594083786, 'x2': 0.4926553964614868},
 ({'mae': 0.8210761838837966}, {'mae': {'mae': nan}}))